In [1]:
import pandas as pd
import numpy as np

In [25]:
!git clone https://github.com/todcru/Sentiment-Analysis

Cloning into 'Sentiment-Analysis'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [28]:
df = pd.read_excel('Sentiment-Analysis/train.xlsx')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [5]:
df.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [31]:
!pip install git+https://github.com/todcru/preprocess_dl.git

  Cloning https://github.com/todcru/preprocess_dl.git to /tmp/pip-req-build-2unhaqgr
  Running command git clone -q https://github.com/todcru/preprocess_dl.git /tmp/pip-req-build-2unhaqgr
  Created wheel for preprocess-dl: filename=preprocess_dl-0.1.3-cp37-none-any.whl size=11576 sha256=5a3b3fbfe5dd166edc58c1282adf2c20432fba7f446a9b3dbf246a8af319061e
  Stored in directory: /tmp/pip-ephem-wheel-cache-x3w51ps4/wheels/62/c9/c3/c691c5319d9b36f3296c514da18b2e3abfe6dc451912e563ac
Successfully built preprocess-dl


In [32]:
import preprocess_dl as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [9]:
df['Reviews'] = df['Reviews'].apply(lambda x: get_clean(x))

In [10]:
df.head()

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [12]:
tfidf = TfidfVectorizer(max_features=5000)
X = df['Reviews']
y = df['Sentiment']

X = tfidf.fit_transform(X)

In [13]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845695 stored elements in Compressed Sparse Row format>

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [17]:
y_pred = clf.predict(X_test)

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [20]:
x = 'this movie is really good. thanks a lot for making it'

x = get_clean(x)
vec = tfidf.transform([x])

In [21]:
vec.shape

(1, 5000)

In [22]:
clf.predict(vec)

array(['pos'], dtype=object)

In [23]:
clf.predict(vec)

array(['pos'], dtype=object)

In [26]:
import pickle
pickle.dump(clf, open('model', 'wb'))
pickle.dump(tfidf, open('tfidf', 'wb'))